In [13]:
!pip3 install tika
!pip install pyLDAvis==3.2
!pip3 install transformers
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
from tika import parser
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
def convertPdf2TxtWithTika(in_pdf_file):
    # Load a file and extract information
    print ("INFO: - reading file = " + in_pdf_file)
    
    raw = parser.from_file(in_pdf_file)
    text = raw['content']
    
    ## Post-processing explained at: 
    # https://medium.com/@justinboylantoomey/fast-text-extraction-with-python-and-tika-41ac34b0fe61
    # Convert to string
    text = str(text)
    # Ensure text is utf-8 formatted
    safe_text = text.encode('utf-8', errors='ignore')
    # Escape any \ issues
    safe_text = str(safe_text).replace('\\', '\\\\').replace('"', '\\"')
    text = text.replace("\n", "")
    text = text.replace("\t", "")
    text = text.replace("\\", "")
    text = text.split(".")
    #text = [x for x in text if not x==""] # data cleaning
    cleaned = []
    for x in text:
      x = x.strip()
      if x=="" or x==" ":
        continue
      cleaned.append(x)
    text = ".".join(cleaned)
    return text

In [69]:
data = convertPdf2TxtWithTika("2022-uosc-securityandfirelreport-1001bcleryreport.pdf")

INFO: - reading file = 2022-uosc-securityandfirelreport-1001bcleryreport.pdf


In [73]:
text = data
print(text)

2022 Annual Security and Fire Safety Report for Years 2019-2021   University of South Carolina Division of Law Enforcement and Safety 2022 Annual Security and Fire Safety Report for Years 2019-2021.University of South Carolina 2022 Annual Security and Fire Safety Report for Calendar Years 2019-2021  Public & Safety Statistics for the Columbia Campus  The Report 5 Annual Disclosure of Crime Statistics and Preparing the Report 6 The Division of Law Enforcement and Safety About the Division 7 Enforcement Authority and Training 8 Interagency Cooperation 8 Reporting Criminal Incidents and Other Emergencies Reporting to the Division of Law Enforcement and Safety 11 Campus Security Authorities 12 Emergency Call Boxes 12 Daily Crime Log 12 Sexual Assault, Domestic Violence, Dating Violence, Relationship Violence and Stalking What is Consent? 14 Definitions of Sexual Assault, Domestic Violence, Dating Violence, and Stalking 15  Interpersonal Violence:  acts of relationship violence including do

In [6]:
# text summarization:

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
import pyLDAvis.gensim as gensimvis
import pyLDAvis
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
from transformers import pipeline
MAX_SIZE = 1024
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
def recursive_summarize(tokens, left, right):
  mid = (left+right)//2
  if (right-left) > MAX_SIZE:
    out_text1 = recursive_summarize(tokens, left, mid+1)[0]["summary_text"]
    out_text2 = recursive_summarize(tokens, mid+1, right)[0]["summary_text"]
    print(out_text1)
    print(out_text2)
    out_text = out_text1 + " " + out_text2
  else:
    out_tokens = tokens[left:right]
    out_text = " ".join(out_tokens)
  max_len = min(len(out_text.split(" ")),512)
  print(left, right)
  print(out_text)
  summary = summarizer(out_text, max_length=max_len, min_length=30, do_sample=False)
  print(summary)
  print("*"*20)
  return summary
def summarize(text):
  return recursive_summarize(text.split(" "), 0, len(text.split(" ")))

print(summarize(text))

0 545
2022 Annual Security and Fire Safety Report for Years 2019-2021 University of South Carolina Division of Law Enforcement and Safety 2022 Annual Security and Fire Safety Report for Years 2019-2021 University of South Carolina 2022 Annual Security and Fire Safety Report for Calendar Years 2019-2021 Public & Safety Statistics for the Columbia Campus The Report 5 Annual Disclosure of Crime Statistics and Preparing the Report 6 The Division of Law Enforcement and Safety About the Division 7 Enforcement Authority and Training 8 Interagency Cooperation 8 Reporting Criminal Incidents and Other Emergencies Reporting to the Division of Law Enforcement and Safety 11 Campus Security Authorities 12 Emergency Call Boxes 12 Daily Crime Log 12 Sexual Assault, Domestic Violence, Dating Violence, Relationship Violence and Stalking What is Consent? 14 Definitions of Sexual Assault, Domestic Violence, Dating Violence, and Stalking 15 Interpersonal Violence: acts of relationship violence including do

KeyboardInterrupt: ignored

In [ ]:
#[{'summary_text': 'The University did not experience any hate crimes, as defined by federal law, in 2019. In 2020 victims at the University experi-enced two hate crimes. In 2021 there was 1 hate crime for Intimidation of a victim under the “On Campus’ category.'}]

In [120]:
# from https://github.com/akashp1712/nlp-akash/blob/master/text-summarization/TF_IDF_Summarization.py
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords

def _create_frequency_table(text_string) -> dict:
    """
    we create a dictionary for the word frequency table.
    For this, we should only use the words that are not part of the stopWords array.
    Removing stop words and making frequency table
    Stemmer - an algorithm to bring words to its root word.
    :rtype: dict
    """
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable


def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix


def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary


def run_summarization(text):
    """
    :param text: Plain summary_text of long article
    :return: summarized summary_text
    """

    '''
    We already have a sentence tokenizer, so we just need 
    to run the sent_tokenize() method to create the array of sentences.
    '''
    # 1 Sentence Tokenize
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
    return summary


if __name__ == '__main__':
    result = run_summarization(text)
    print(result)

 2022 Annual Security and Fire Safety Report for Years 2019-2021   University of South Carolina Division of Law Enforcement and Safety 2022 Annual Security and Fire Safety Report for Years 2019-2021.University of South Carolina 2022 Annual Security and Fire Safety Report for Calendar Years 2019-2021  Public & Safety Statistics for the Columbia Campus  The Report 5 Annual Disclosure of Crime Statistics and Preparing the Report 6 The Division of Law Enforcement and Safety About the Division 7 Enforcement Authority and Training 8 Interagency Cooperation 8 Reporting Criminal Incidents and Other Emergencies Reporting to the Division of Law Enforcement and Safety 11 Campus Security Authorities 12 Emergency Call Boxes 12 Daily Crime Log 12 Sexual Assault, Domestic Violence, Dating Violence, Relationship Violence and Stalking What is Consent? : offers bicycle registration.• Women’s Self Defense Workshops: increases recognition and response to potentially dangerous situations, in-creasing confi

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

In [ ]:
# Sentiment analysis 
sentences = text.split(".")
print(len(sentences))
cleaned = []
for sentence in sentences:
  words = sentence.split(" ")
  words = [re.sub(r'[^A-Za-z0-9 ]+', '', word) for word in words]
  if len(words) > 512:
    cleaned.append(" ".join(words[:512]))
    maxlen = min(1024, len(words))
    cleaned.append(" ".join(words[512:maxlen]))
  else:
    cleaned.append(sentence)
print(cleaned)
for i,sentence in enumerate(cleaned):
  try:
    results = classifier([sentence])
    for result in results:
        if round(result['score'], 4) > 0.999:
          print(cleaned[i])
          print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
  except:
    print("error for sentence:", cleaned[i])
    continue

print(i, len(cleaned))

493
['2022 Annual Security and Fire Safety Report for Years 2019-2021   University of South Carolina Division of Law Enforcement and Safety 2022 Annual Security and Fire Safety Report for Years 2019-2021', 'University of South Carolina 2022 Annual Security and Fire Safety Report for Calendar Years 2019-2021  Public & Safety Statistics for the Columbia Campus  The Report 5 Annual Disclosure of Crime Statistics and Preparing the Report 6 The Division of Law Enforcement and Safety About the Division 7 Enforcement Authority and Training 8 Interagency Cooperation 8 Reporting Criminal Incidents and Other Emergencies Reporting to the Division of Law Enforcement and Safety 11 Campus Security Authorities 12 Emergency Call Boxes 12 Daily Crime Log 12 Sexual Assault, Domestic Violence, Dating Violence, Relationship Violence and Stalking What is Consent? 14 Definitions of Sexual Assault, Domestic Violence, Dating Violence, and Stalking 15  Interpersonal Violence:  acts of relationship violence inc

In [122]:
# Sentiment analysis 
sentences = text.split(".")
print(len(sentences))
cleaned = []
for sentence in sentences:
  words = sentence.split(" ")
  words = [re.sub(r'[^A-Za-z0-9 ]+', '', word) for word in words]
  cleaned.append(sentence)

pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist


from textblob import TextBlob
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
    # function to calculate polarity
    def getPolarity(review):
        return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

def get_sentiment(word):
  pos = token_stop_pos(word)
  wordnet_lemmatizer = WordNetLemmatizer()
  lemma_rew = " "
  for word, pos in pos:
    if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
    else:
        if pos=="CD":
          continue
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma

  polarity = getSubjectivity(lemma_rew)
  score = analysis(polarity)
  return (polarity, score)

for sentence in cleaned:
  print(get_sentiment(sentence), sentence)

493
(0.0, 'Neutral') 2022 Annual Security and Fire Safety Report for Years 2019-2021   University of South Carolina Division of Law Enforcement and Safety 2022 Annual Security and Fire Safety Report for Years 2019-2021
(0.5083333333333333, 'Positive') University of South Carolina 2022 Annual Security and Fire Safety Report for Calendar Years 2019-2021  Public & Safety Statistics for the Columbia Campus  The Report 5 Annual Disclosure of Crime Statistics and Preparing the Report 6 The Division of Law Enforcement and Safety About the Division 7 Enforcement Authority and Training 8 Interagency Cooperation 8 Reporting Criminal Incidents and Other Emergencies Reporting to the Division of Law Enforcement and Safety 11 Campus Security Authorities 12 Emergency Call Boxes 12 Daily Crime Log 12 Sexual Assault, Domestic Violence, Dating Violence, Relationship Violence and Stalking What is Consent? 14 Definitions of Sexual Assault, Domestic Violence, Dating Violence, and Stalking 15  Interpersonal

In [7]:
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text

tokenized_data = [clean_text(text)]
dictionary = corpora.Dictionary(tokenized_data)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
# Build the LSI model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
vis_data1 = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data1)

<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<ipython-input-7-b6548a97e8f5>:6: DeprecationWarning: invalid escape sequence \-
  cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
